In [3]:
import json

In [15]:
class Node(json.JSONEncoder):
    def __init__(self, data):
        self.isRoot = False
        self.parent = None
        self.children = []
        self.data = data
        self.layer = None
        self.index = None
    
    def addChild(self, child):
        if child.layer != None and self.layer > child.layer:
            return False
        
        child.parent = self
        self.children.append(child)
        child.layer = self.layer + 1
        return True
    
    def makeRoot(self):
        self.layer = 0
        self.isRoot = True
    
    def stringify(self):
        return json.dumps(self.__dict__)
    
    def __repr__(self):
        return str(self.index)
    
    def getJSON(self, keyname):
        if keyname == 'children':
            return [int(repr(x)) for x in self.children]
        elif keyname == 'parent':
            return None if self.parent is None else int(repr(self.parent))
        else:
            return self.__dict__[keyname]
    
    def default(self, obj):
        return {x:obj.getJSON(x) for x in obj.__dict__.keys()}
        
class NodeList(json.JSONEncoder):
    def __init__(self, array=[]):
        for i,x in enumerate(array):
            x.index = i
        self.nodes = array
        
    def add(self, node):
        node.index = len(self.nodes)
        self.nodes.append(node)
        
    def remove(self, index):
        del self.nodes[index]
        
    def default(self, obj):
        return [x.default(x) for x in obj.nodes]

In [16]:
functions = []

a = Node("f(x)=2x")
a.makeRoot()
b = Node("x")
a.addChild(b)

arr = NodeList(array=[a,b])
functions.append(arr.default(arr))

c = Node("g(x,y)=x^2+y^2")
c.makeRoot()
d = Node("x")
c.addChild(d)
e = Node("y")
c.addChild(e)

functions.append(arr.default(NodeList(array=[c,d,e])))





In [17]:
functions

[[{'children': [1],
   'data': 'f(x)=2x',
   'index': 0,
   'isRoot': True,
   'layer': 0,
   'parent': None},
  {'children': [],
   'data': 'x',
   'index': 1,
   'isRoot': False,
   'layer': 1,
   'parent': 0}],
 [{'children': [1, 2],
   'data': 'g(x,y)=x^2+y^2',
   'index': 0,
   'isRoot': True,
   'layer': 0,
   'parent': None},
  {'children': [],
   'data': 'x',
   'index': 1,
   'isRoot': False,
   'layer': 1,
   'parent': 0},
  {'children': [],
   'data': 'y',
   'index': 2,
   'isRoot': False,
   'layer': 1,
   'parent': 0}]]

In [18]:
json.dumps([x for x in functions])

'[[{"parent": null, "index": 0, "isRoot": true, "layer": 0, "data": "f(x)=2x", "children": [1]}, {"parent": 0, "index": 1, "isRoot": false, "layer": 1, "data": "x", "children": []}], [{"parent": null, "index": 0, "isRoot": true, "layer": 0, "data": "g(x,y)=x^2+y^2", "children": [1, 2]}, {"parent": 0, "index": 1, "isRoot": false, "layer": 1, "data": "x", "children": []}, {"parent": 0, "index": 2, "isRoot": false, "layer": 1, "data": "y", "children": []}]]'

In [19]:
with open('functions.json', 'w+') as f:
    f.write(json.dumps([x for x in functions]))